In [202]:
! pip install langchain faiss-cpu requests beautifulsoup4
! pip install euriai

In [203]:
from langchain.embeddings.base import Embeddings
from langchain.llms.base import LLM
from typing import List, Optional, Dict, Any
import requests
import numpy as np
import faiss
import os
import json

# Fix: Correct way to import and load environment variables
from dotenv import load_dotenv
load_dotenv()  # This loads environment variables from a .env file


True

In [204]:
# Using requests library for embeddings
import requests
import numpy as np

def generate_embeddings(text: str):
    url = "https://api.euron.one/api/v1/euri/alpha/embeddings"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {os.getenv('EURI_API_KEY')}"  # Ensure you set this environment variable"
    }
    payload = {
        "input": text,
        "model": "text-embedding-3-small"
    }

    response = requests.post(url, headers=headers, json=payload)
    data = response.json()
    
    # Convert to numpy array for vector operations
    embedding = np.array(data['data'][0]['embedding'])
    
    
    return embedding

In [205]:
from euriai import EuriaiClient

def generate_response(prompt):
    client = EuriaiClient(
        api_key=os.getenv('EURI_API_KEY'),  # Ensure you set this environment variable
        model="gpt-4.1-nano"
    )

    response = client.generate_completion(
        prompt=prompt,
        temperature=0.7,
        max_tokens=300
    )

    print(response)
    return response['choices'][0]["message"]["content"]

In [206]:
! pip install langchain-community 

In [207]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

document = TextLoader("knowledgebase.txt", encoding="utf-8").load()  # Load the text file
document


[Document(metadata={'source': 'knowledgebase.txt'}, page_content="\nMaking an Impact\nHelping Millions of Students Succeed\nSudhanshu's commitment to affordable education wasn't just a business strategy—it was his life's mission. Over the years, iNeuron has helped over 1.5 million students from 34+ countries, providing them with the skills they need to succeed in today's competitive job market. Many of these students, like Sudhanshu himself, came from disadvantaged backgrounds. They saw iNeuron as a lifeline—an opportunity to rise above their circumstances.\n\nIn 2022, iNeuron was acquired by PhysicsWallah in a deal worth ₹250 crore. While this acquisition was a significant milestone, Sudhanshu remained focused on his mission. Even after the acquisition, iNeuron continued to offer some of the most affordable and accessible tech courses in the world.\n\nThe Entrepreneur and Teacher: Sudhanshu's Dual Legacy\nSudhanshu's journey isn't just one of entrepreneurial success; it's also a story

In [208]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # Adjust chunk size as needed
    chunk_overlap=30  # Adjust overlap as needed
)
docs  = text_splitter.split_documents(document)  # Split the document into chunks

In [209]:
len(docs)

58

In [210]:
docs 

[Document(metadata={'source': 'knowledgebase.txt'}, page_content='Making an Impact\nHelping Millions of Students Succeed'),
 Document(metadata={'source': 'knowledgebase.txt'}, page_content="Sudhanshu's commitment to affordable education wasn't just a business strategy—it was his life's mission. Over the years, iNeuron has helped over 1.5 million students from 34+ countries, providing them with the skills they need to succeed in today's competitive job market. Many of these students,"),
 Document(metadata={'source': 'knowledgebase.txt'}, page_content='Many of these students, like Sudhanshu himself, came from disadvantaged backgrounds. They saw iNeuron as a lifeline—an opportunity to rise above their circumstances.'),
 Document(metadata={'source': 'knowledgebase.txt'}, page_content='In 2022, iNeuron was acquired by PhysicsWallah in a deal worth ₹250 crore. While this acquisition was a significant milestone, Sudhanshu remained focused on his mission. Even after the acquisition, iNeuron co

In [211]:
texts = [doc.page_content for doc in docs]
texts

['Making an Impact\nHelping Millions of Students Succeed',
 "Sudhanshu's commitment to affordable education wasn't just a business strategy—it was his life's mission. Over the years, iNeuron has helped over 1.5 million students from 34+ countries, providing them with the skills they need to succeed in today's competitive job market. Many of these students,",
 'Many of these students, like Sudhanshu himself, came from disadvantaged backgrounds. They saw iNeuron as a lifeline—an opportunity to rise above their circumstances.',
 'In 2022, iNeuron was acquired by PhysicsWallah in a deal worth ₹250 crore. While this acquisition was a significant milestone, Sudhanshu remained focused on his mission. Even after the acquisition, iNeuron continued to offer some of the most affordable and accessible tech courses in the world.',
 "The Entrepreneur and Teacher: Sudhanshu's Dual Legacy",
 "Sudhanshu's journey isn't just one of entrepreneurial success; it's also a story of dedication to teaching. Th

In [212]:
import time 
import numpy as np 

embeddings = []

for i, embedding in enumerate(embeddings):
    try:
        print(f"Processing chunk {i+1}/{len(texts)}...")
        embeddings.append(embedding)
        time.sleep(1)  # Sleep to avoid hitting API rate limits
    except Exception as e:
        print(f"Error processing chunk {i+1}: {e}")
            

In [213]:
embeddings

[]

In [214]:
texts

['Making an Impact\nHelping Millions of Students Succeed',
 "Sudhanshu's commitment to affordable education wasn't just a business strategy—it was his life's mission. Over the years, iNeuron has helped over 1.5 million students from 34+ countries, providing them with the skills they need to succeed in today's competitive job market. Many of these students,",
 'Many of these students, like Sudhanshu himself, came from disadvantaged backgrounds. They saw iNeuron as a lifeline—an opportunity to rise above their circumstances.',
 'In 2022, iNeuron was acquired by PhysicsWallah in a deal worth ₹250 crore. While this acquisition was a significant milestone, Sudhanshu remained focused on his mission. Even after the acquisition, iNeuron continued to offer some of the most affordable and accessible tech courses in the world.',
 "The Entrepreneur and Teacher: Sudhanshu's Dual Legacy",
 "Sudhanshu's journey isn't just one of entrepreneurial success; it's also a story of dedication to teaching. Th

In [215]:
valid_texts = []
for i, text in enumerate(texts[0:8]):
    clean_text = text.strip()
    if not clean_text:
        print(f"Skipping empty text at index {i}")
        continue
    emb = generate_embeddings(text)
    embeddings.append(emb)
    valid_texts.append(text)
    print(f"Processed {i+1}/{len(texts)}: {text[:30]}... -> {emb[:5]}")


Processed 1/58: Making an Impact
Helping Milli... -> [ 0.03395454 -0.0469909   0.03878181  0.07887272  0.02695909]
Processed 2/58: Sudhanshu's commitment to affo... -> [-0.01364236  0.00639423  0.05721121  0.02440282  0.04653747]
Processed 3/58: Many of these students, like S... -> [-0.02204193 -0.00722562  0.00345266  0.03793968  0.04818918]
Processed 4/58: In 2022, iNeuron was acquired ... -> [-0.04330519 -0.05195538  0.02764536  0.0217475   0.03644469]
Processed 5/58: The Entrepreneur and Teacher: ... -> [0.01671011 0.01011956 0.04287479 0.01667061 0.00875009]
Processed 6/58: Sudhanshu's journey isn't just... -> [-0.00970297 -0.00214104  0.0598214   0.02851626  0.0276796 ]
Processed 7/58: in Big Data, Data Science, or ... -> [-0.00568316 -0.01421538  0.03512035 -0.01586388  0.05972848]
Processed 8/58: His commitment to affordable e... -> [ 0.01117547 -0.04308555  0.04391897  0.05712749  0.01579719]


In [216]:
embeddings

[array([ 0.03395454, -0.0469909 ,  0.03878181, ..., -0.02398636,
        -0.01851818,  0.01633636]),
 array([-0.01364236,  0.00639423,  0.05721121, ..., -0.00413274,
        -0.00975312, -0.00021493]),
 array([-0.02204193, -0.00722562,  0.00345266, ..., -0.00756314,
        -0.02643654, -0.00205438]),
 array([-0.04330519, -0.05195538,  0.02764536, ..., -0.03267549,
         0.01820879,  0.03354321]),
 array([ 0.01671011,  0.01011956,  0.04287479, ..., -0.04371754,
         0.01135734, -0.00684733]),
 array([-0.00970297, -0.00214104,  0.0598214 , ..., -0.01701214,
        -0.01090567,  0.00101315]),
 array([-0.00568316, -0.01421538,  0.03512035, ..., -0.00705393,
         0.00747801,  0.00206959]),
 array([ 0.01117547, -0.04308555,  0.04391897, ...,  0.00480482,
        -0.02457341,  0.01011475])]

In [217]:
import faiss
from langchain.vectorstores.faiss import FAISS
from langchain_core.documents import Document



In [218]:
dimensions = embeddings[0].shape[0]
faiss_index = faiss.IndexFlatL2(dimensions)  # Create a FAISS index for L2 distance
  # Add embeddings to the index


In [219]:
faiss_index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001ACCDD01980> >

In [220]:
faiss_index.add(np.array(embeddings).astype('float32'))  

In [221]:
docs[0:10]


[Document(metadata={'source': 'knowledgebase.txt'}, page_content='Making an Impact\nHelping Millions of Students Succeed'),
 Document(metadata={'source': 'knowledgebase.txt'}, page_content="Sudhanshu's commitment to affordable education wasn't just a business strategy—it was his life's mission. Over the years, iNeuron has helped over 1.5 million students from 34+ countries, providing them with the skills they need to succeed in today's competitive job market. Many of these students,"),
 Document(metadata={'source': 'knowledgebase.txt'}, page_content='Many of these students, like Sudhanshu himself, came from disadvantaged backgrounds. They saw iNeuron as a lifeline—an opportunity to rise above their circumstances.'),
 Document(metadata={'source': 'knowledgebase.txt'}, page_content='In 2022, iNeuron was acquired by PhysicsWallah in a deal worth ₹250 crore. While this acquisition was a significant milestone, Sudhanshu remained focused on his mission. Even after the acquisition, iNeuron co

In [222]:
generate_response("Tell me somethign about sudhanshu ")

{'id': 'chatcmpl-BiiZ6lJ4VxjV8WiQ6dtMStKO5MA2b', 'object': 'chat.completion', 'created': 1749997440, 'model': 'gpt-4.1-nano', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Sure! Could you please specify which Sudhanshu you're referring to? It’s a common name, and I want to make sure I provide you with accurate information about the right person."}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 18, 'completion_tokens': 38, 'total_tokens': 56}}


"Sure! Could you please specify which Sudhanshu you're referring to? It’s a common name, and I want to make sure I provide you with accurate information about the right person."

In [223]:
from langchain.chains import RetrievalQA

In [224]:
documents = [Document(page_content=text) for text in texts[:len(embeddings)]]

In [225]:
embeddings 

[array([ 0.03395454, -0.0469909 ,  0.03878181, ..., -0.02398636,
        -0.01851818,  0.01633636]),
 array([-0.01364236,  0.00639423,  0.05721121, ..., -0.00413274,
        -0.00975312, -0.00021493]),
 array([-0.02204193, -0.00722562,  0.00345266, ..., -0.00756314,
        -0.02643654, -0.00205438]),
 array([-0.04330519, -0.05195538,  0.02764536, ..., -0.03267549,
         0.01820879,  0.03354321]),
 array([ 0.01671011,  0.01011956,  0.04287479, ..., -0.04371754,
         0.01135734, -0.00684733]),
 array([-0.00970297, -0.00214104,  0.0598214 , ..., -0.01701214,
        -0.01090567,  0.00101315]),
 array([-0.00568316, -0.01421538,  0.03512035, ..., -0.00705393,
         0.00747801,  0.00206959]),
 array([ 0.01117547, -0.04308555,  0.04391897, ...,  0.00480482,
        -0.02457341,  0.01011475])]

In [227]:
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.document import Document
document = [Document(page_content=text) for text in texts[:len(embeddings)]]

import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

faiss_index = faiss.IndexFlatL2(dimensions)
vector_store = FAISS(
    embedding_function=generate_embeddings,
    index=faiss_index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)
vector_store.add_documents(document)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


['795691ff-3939-4350-b6bf-b6c6e95571b7',
 'f8516ee7-54f8-4f6f-ab7b-0709ba54adce',
 '6346ae4c-0dc8-44a8-be6a-5b06e9e8bf35',
 'ffed5d7f-8a3a-4888-927a-0ea9477ec0b0',
 'e4ec61d5-624a-406f-82a0-ba2e3c617fe0',
 'c83bcbc7-b569-4549-ad46-d2b0f9376b80',
 '7aa35473-7d4b-4c17-944a-5233f4515ac1',
 'aa617731-ee5a-4d50-a83d-75888ece325f']

In [228]:
vector_store.save_local("faiss_index")

In [229]:
retriver = vector_store.as_retriever( search_kwargs={"k": 3})

In [231]:
from euriai import EuriaiLangChainLLM

llm = EuriaiLangChainLLM(
    api_key=os.getenv('EURI_API_KEY'),  # Ensure you set this environment variable
    model="gpt-4.1-nano",
    temperature=0,
    max_tokens=300
)

print(llm.invoke("Write a poem about time travel."))

Through whispers of the ticking clock we go,  
A journey woven in the threads of flow,  
Beyond the present's fleeting, fragile glow,  
Where past and future intertwine and grow.

We chase the shadow of a moment lost,  
A whisper of what once was, at what cost?  
In loops of time, our memories embossed,  
A tapestry of lines, both gained and tossed.

The seconds stretch to years, then vanish light,  
As we traverse corridors in endless night,  
Seeking the dawn that breaks beyond our sight,  
In realms where yesterday and tomorrow unite.

Oh, boundless voyage through the sands of time,  
A dance of moments, silent, so divine,  
To hold eternity within a rhyme,  
And glimpse the future's vast design.


In [232]:
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever = retriver)

In [ ]:
rag_chain.run("What is the capital of France?")

In [235]:
rag_chain("Whow is Sudhanshu?")

C:\Users\Deepu\AppData\Local\Temp\ipykernel_4488\1975610730.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  rag_chain("Whow is Sudhanshu?")


{'query': 'Whow is Sudhanshu?',
 'result': 'Sudhanshu is an educator and entrepreneur dedicated to making complex subjects like Big Data and Data Science accessible to learners at all levels. He is known for his commitment to affordable education and has helped many students improve their skills, secure jobs, and transform their lives through his teaching.'}

In [237]:
rag_chain.run("give me the company where sudhanshu is working")

'The company where Sudhanshu is working is iNeuron.'

In [238]:
generate_response("Who is sudhasnhu?")

{'id': 'chatcmpl-BiiiJFl9oq87fpiAoL3C1skxVpgPO', 'object': 'chat.completion', 'created': 1749998011, 'model': 'gpt-4.1-nano', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Sudhasnhu appears to be a username or online pseudonym, but there is no widely known public figure, celebrity, or notable individual by that name based on available information up to October 2023. If you can provide more context or specify the field or platform where you've encountered this name, I may be able to assist you better."}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 14, 'completion_tokens': 69, 'total_tokens': 83}}


"Sudhasnhu appears to be a username or online pseudonym, but there is no widely known public figure, celebrity, or notable individual by that name based on available information up to October 2023. If you can provide more context or specify the field or platform where you've encountered this name, I may be able to assist you better."